<a href="https://colab.research.google.com/github/DerekLiu35/GeneRE/blob/main/RelationExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/michiyasunaga/LinkBERT

Cloning into 'LinkBERT'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 42 (delta 11), reused 42 (delta 11), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [ ]:
!conda create -n linkbert python=3.8
!source activate linkbert
!pip install torch==1.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install transformers==4.9.1 datasets==1.11.0 fairscale==0.4.0 wandb sklearn seqeval

/bin/bash: conda: command not found
/bin/bash: activate: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 169.5 MB/s eta 0:00:01tcmalloc: large alloc 1821442048 bytes == 0x3228000 @  0x7f702feb71e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 GB 208.2 MB/s eta 0:00:01tcmalloc: large alloc 2276802560 bytes == 0x6fb38000 @  0x7f702feb8615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4

In [ ]:
!wget https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip

--2023-01-13 04:19:25--  https://nlp.stanford.edu/projects/myasu/LinkBERT/data.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458233246 (437M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 437.00M  17.1MB/s    in 30s     

2023-01-13 04:19:56 (14.6 MB/s) - ‘data.zip’ saved [458233246/458233246]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/tokcls/
   creating: data/tokcls/BC5CDR-disease_hf/
  inflating: data/tokcls/BC5CDR-disease_hf/test.json  
  inflating: data/tokcls/BC5CDR-disease_hf/train.json  
  inflating: data/tokcls/BC5CDR-disease_hf/dev.json  
   creating: data/tokcls/BC2GM_hf/
  inflating: data/tokcls/BC2GM_hf/train.json  
  inflating: data/tokcls/BC2GM_hf/dev.json  
  inflating: data/tokcls/BC2GM_hf/test.json  
   creating: data/tokcls/ebmnlp_hf/
  inflating: data/tokcls/ebmnlp_hf/test.json  
  inflating: data/tokcls/ebmnlp_hf/train.json  
  inflating: data/tokcls/ebmnlp_hf/dev.json  
   creating: data/tokcls/NCBI-disease_hf/
  inflating: data/tokcls/NCBI-disease_hf/test.json  
  inflating: data/tokcls/NCBI-disease_hf/dev.json  
  inflating: data/tokcls/NCBI-disease_hf/train.json  
   creating: data/tokcls/JNLPBA_hf/
  inflating: data/tokcls/JNLPBA_hf/dev.json  
  inflating: data/tokcls/JNLPBA_hf/train.json  
  inflating: data/tokcls/JNLPBA_hf/test.json  

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%%shell

cd LinkBERT/src

MODEL=BioLinkBERT-base
MODEL_PATH=michiyasunaga/$MODEL

task=NCBI-disease_hf
datadir=../../data/tokcls/$task
outdir=runs/$task/$MODEL
mkdir -p $outdir
python3 -u tokcls/run_ner.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train.json --validation_file $datadir/dev.json --test_file $datadir/test.json \
  --do_train --do_eval --do_predict \
  --per_device_train_batch_size 32 --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 5e-5 --warmup_ratio 0.1 --num_train_epochs 20 --max_seq_length 512 \
  --save_strategy no --evaluation_strategy no --output_dir $outdir --overwrite_output_dir \
  |& tee $outdir/log.txt &

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=runs/NCBI-disease_hf/BioLinkBERT-base/runs/Jan13_04-32-57_6d797a5a4c65,
logging_first_step=False,
logging_steps=500,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=S

In [ ]:
%%shell

cd LinkBERT/src

MODEL=BioLinkBERT-base
MODEL_PATH=michiyasunaga/$MODEL

task=GAD_hf
datadir=../../data/seqcls/$task
outdir=runs/$task/$MODEL
mkdir -p $outdir
python3 -u seqcls/run_seqcls.py --model_name_or_path $MODEL_PATH \
  --train_file $datadir/train.json --validation_file $datadir/dev.json --test_file $datadir/test.json \
  --do_train --do_eval --do_predict --metric_name PRF1 \
  --per_device_train_batch_size 32 --gradient_accumulation_steps 1 --fp16 \
  --learning_rate 3e-5 --num_train_epochs 10 --max_seq_length 256 \
  --save_strategy no --evaluation_strategy no --output_dir $outdir --overwrite_output_dir \
  |& tee $outdir/log.txt &

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=3e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=runs/GAD_hf/BioLinkBERT-base/runs/Jan13_03-07-33_ddf17b5aa518,
logging_first_step=False,
logging_steps=500,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerT